<h1 style='text-align:center;' > Basic Data Visualization with Bokeh <h1>

## Onde procurar ajuda 
- Tutorial básico:
   https://bokeh.pydata.org/en/latest/docs/user_guide.html
- Explorando possibilidades:
   https://bokeh.pydata.org/en/latest/docs/gallery.html
- Tutorial interativo:
   https://mybinder.org/v2/gh/bokeh/bokeh-notebooks/master?filepath=tutorial%2F00%20-%20Introduction%20and%20Setup.ipynb
   
- Exemplo:
https://felipeaoli.github.io/qlf_tests/

In [3]:
import numpy as np

In [4]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, Whisker
from bokeh.plotting import Figure, output_notebook, show
from bokeh.models import HoverTool

## Formatação  & Estilo

In [5]:
def style(p):
    # Title 
    p.title.align = 'center'
    p.title.text_font_size = '20pt'
    p.title.text_font = 'serif'

    # Axis titles
    p.xaxis.axis_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_style = 'bold'

    # Tick labels
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'
    p.xaxis.ticker = np.arange(0,1.5, 0.3)
    return p


In [31]:
my_tooltip = """
            <div>
                <div>
                    <span style="font-size: 12px; font-weight: bold; color: #303030;">Name: </span>
                    <span style="font-size: 13px; color: #515151">@name</span>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold; color: #303030;">redshift: </span>
                    <span style="font-size: 13px; color: #515151;">@z</span>
                </div>
                <div>
                    <span style="font-size: 12px; font-weight: bold; color: #303030;">Distance Modulus: </span>
                    <span style="font-size: 13px; color: #515151;">@mu</span>
                </div>
            </div>
        """

def custom_cosmo_tooltip(Om,wm):
    return f""" 
            <div>
                <div>
                    <span style="font-size: 12px; font-weight: bold; color: #303030;">Cosmology: ( Ωm,  ω ): </span>
                    <span style="font-size: 13px; color: #515151"> {Om:.2f}, {wm: .2f}</span>
                </div>
            </div>
            """
hover = HoverTool(tooltips= my_tooltip)


## Loading data 

In [32]:
# Loading Sn Names!
with open('data/SCPUnion2.1_mu_vs_z.txt') as f:
    c = [ line.split('\t')[0] for line in f.readlines()[5:] ]

    
# Loading data!
data = np.loadtxt('data/SCPUnion2.1_mu_vs_z.txt', skiprows=5, usecols=[1,2,3,4]).T

## error bars
lower = data[1] -data[2]
upper = data[1] +data[2]


## Curvas teóricas 

In [33]:
# Preparing Theoretical Data 
import pyccl as ccl
z_th  = np.linspace(0.01, 1.5, 100)
a_th  = 1/(1+z_th)


cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.7, A_s=2.1e-9, n_s=0.96, 
                      Neff=3.046, Omega_k=0.)

mu_th = ccl.distance_modulus(cosmo, a_th)

cosmo = ccl.Cosmology(Omega_c=0.99, Omega_b=0.045, h=0.7, A_s=2.1e-9, n_s=0.96, 
                      Neff=3.046, Omega_k=0., w0=-1)
mu_th2 = ccl.distance_modulus(cosmo, a_th)

cosmo = ccl.Cosmology(Omega_c=0.27, Omega_b=0.045, h=0.7, A_s=2.1e-9, n_s=0.96, 
                      Neff=3.046, Omega_k=0., w0=-1.3)
mu_th3 = ccl.distance_modulus(cosmo, a_th)


## Bokeh

In [40]:
output_notebook()

# 1)  Organizando o dados em ColumnDataSources
source = ColumnDataSource( data = dict( name=c , 
                                       mu=data[1], 
                                       z=data[0], 
                                       base=data[0], 
                                       lower=lower, 
                                       upper=upper  ))

source_th=ColumnDataSource(data = dict( z=z_th, 
                                        mu=mu_th,
                                        mu2=mu_th2,
                                        mu3 = mu_th3,
                                        name = ['Model 1']*len(z_th)))

# 2) Preparando a Figura
plot = Figure(  title="Union 2.1", 
                plot_width=800, plot_height=400, 
                x_axis_label="z",
                y_axis_label="μ",
                x_range=(0,1.5),
                tools=['pan, wheel_zoom, box_select, crosshair,reset'])


    
# 3) Preparando os glyphs     
# 3.1) theory:
plot2= plot.line('z', 'mu', source=source_th, 
           line_color='red', line_width=3)
plot.add_tools(HoverTool(renderers=[plot2], tooltips=custom_cosmo_tooltip( 0.27, -1))) #,mode='hline'))

plot2= plot.line('z', 'mu2', source=source_th, 
           line_color='orange', line_width=3)
plot.add_tools(HoverTool(renderers=[plot2], tooltips= custom_cosmo_tooltip(0.99, -1)  )) #,mode='hline'))

plot2= plot.line('z', 'mu3', source=source_th, 
           line_color='green', line_width=3)
plot.add_tools(HoverTool(renderers=[plot2], tooltips= custom_cosmo_tooltip(0.27, -1.3)  )) #,mode='hline'))



# 3.2) Data:
plot1 = plot.circle('z', 'mu', source=source, 
            size=6, line_alpha=0.9,
            fill_color="dodgerblue", line_color="black", fill_alpha=0.5, hover_fill_color="red")


# 4) Layout:
plot.add_layout(
    Whisker(source=source, base="z", 
            upper="upper", lower="lower", 
            line_alpha=0.2, line_color='black', line_cap='square') )
plot.add_tools(HoverTool(renderers=[plot1], tooltips=my_tooltip, mode='mouse'))


# Adicionando a Formatação 
style(plot)
layout = plot
show(layout)

Loading BokehJS ...

## Exercício 1: Plotar resíduos! 

( i.e.,  Dados / Curva_fiducial ) assumindo o modelo fiducial como $\Lambda$ CDM